In [1]:
import os
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
#MODEL = "llama2"
MODEL = "tinyllama"

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)



In [2]:
#langchain to load chroma db data
from langchain_community.embeddings import HuggingFaceEmbeddings
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
from langchain.vectorstores import Chroma
chroma_db = Chroma(persist_directory="./chroma_db", 
                   embedding_function=embed_model,
                   collection_name="DB_collection")
retriever = chroma_db.as_retriever(search_kwargs={"k": 5})
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001D595C5DFC0>, search_kwargs={'k': 5})

In [3]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.globals import set_debug
from langchain.prompts import PromptTemplate
set_debug(True)

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".
Context: {context}
Question: {input}
"""
prompt = PromptTemplate.from_template(template)


combine_docs_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

#Invoke the retrieval chain
response=retrieval_chain.invoke({"input":"What is data lake?"})

#Print the answer to the question
print(response["answer"])



[chain/start] [1:chain:retrieval_chain] Entering Chain run with input:
{
  "input": "What is data lake?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context>] Entering Chain run with input:
{
  "input": "What is data lake?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context>] Entering Chain run with input:
{
  "input": "What is data lake?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chain:retrieve_documents] Entering Chain run with input:
{
  "input": "What is data lake?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chain:retrieve_documents > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": "What is data lake?"
}
[chain/end] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chain:retrieve_documents > 

In [7]:
!pip freeze > requirements.txt